Generate datasets:
* 100 signals with tau 100ms and length 30 min

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from scipy.optimize import curve_fit

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
#sys.path.append('C:\\Users\\ipochino\\.conda\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') 
import abcTau

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

### Generate time series

In [3]:
save_folder = project_folder_path + 'results\\synthetic_data\\dataset_long\\'

n_signals = 1

In [24]:
tau = np.array([100])
D = 1/tau
deltaT = 1 # 1ms
T = 30*60*1000 
numTrials = 1

data_mean = 0.01 # average of firing rate
data_var = 0.02 # variance of firing rate

binSize = 50 # bins size for binning the data for calculating acf
binsData =  np.arange(0, T + binSize, binSize)
numBinData = len(binsData)-1
#print(binsData)
print(numBinData)

num_lags = 20
maxTimeLag = int(T/binSize)
print(maxTimeLag)

36000
36000


In [4]:
save_folder

'E:\\work\\q_backup_06_03_2025\\projects\\isttc\\results\\synthetic_data\\dataset_long\\'

In [25]:
for signal_ in range(n_signals):
    ###############
    # OU process 
    ou = abcTau.OU_gen(tau, D, deltaT, T, numTrials)
    print('ou.shape: ', ou.shape)
    
    ou_check = np.max(ou)
    print(ou_check)
    # adjust OU process - scale, shift 
    ou_std = np.sqrt(data_var)
    ou_mean = data_mean
    ou_all = ou_std * ou + ou_mean # this is scale and shift, where is rectify? - no rectify here, this will come for the Poisson rate
    print('ou_all.shape: ', ou_all.shape)
    
    ###############
    # One tau UO with Poisson spike count
    ou_std =  np.sqrt(data_var - data_mean)/deltaT # law of total variance  
    ou_mean = data_mean/deltaT # law of total expectation
    
    # fit mean and var
    ou_poisson = ou_std * ou + ou_mean
    ou_poisson[ou_poisson < 0] = 0 # rectifying
    
    # bin rate and generate spikes
    numBin = int(T/deltaT)
    ou_poisson_rate_ = abcTau.binData(ou_poisson, [numTrials,numBin]) * deltaT
    ou_poisson_rate = np.random.poisson(ou_poisson_rate_)
    
    ###############
    # Going from rate to spikes
    spikeTrain = np.zeros(ou_poisson_rate.shape)
    print(spikeTrain.shape)
    
    for i in range(ou_poisson_rate.shape[0]):
        spikeTrain[i,:] = [1 if ou_poisson_rate[i,j] > np.random.random() else 0 for j in range(ou_poisson_rate.shape[1])]
    
    spike_times = np.where(np.squeeze(spikeTrain) == 1)[0]
    print('N spikes {}'.format(spike_times.shape))
    
    # bin for ACF calculation
    ou_spiketrain_binned = abcTau.binData(spikeTrain, [numTrials, numBinData]) * deltaT
    print(ou_spiketrain_binned.shape)
    
    # store
    np.save(save_folder + 'spike_times_' + str(signal_) + '.npy', spike_times)
    np.save(save_folder + 'spike_times_binned_' + str(signal_) + '.npy', ou_spiketrain_binned)

ou.shape:  (1, 1800000)
4.5959717494259
ou_all.shape:  (1, 1800000)
(1, 1800000)
N spikes (76596,)
(1, 36000)


In [30]:
len(spike_times/)

76596

In [21]:
np.linspace(0,19,20)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19.])

In [29]:
def func_exp_abc_like(x, a, tau):
    return a * np.exp(-x/tau) 

disp = None # put the dispersion parameter if computed with grid-search
#maxTimeLag = 20 # only used when using autocorrelation for summary statistics
summStat_metric = 'comp_cc'
ifNorm = True # if normalize the autocorrelation or PSD
data_sumStat, data_mean, data_var, T, numTrials =  abcTau.preprocessing.extract_stats(ou_poisson_rate, deltaT, binSize,
                                                                                  summStat_metric, ifNorm, maxTimeLag=1000)

# summStat_metric = 'comp_cc'
# ifNorm = True # if normalize the autocorrelation or PSD
# ou_all_data_sumStat, ou_all_data_mean, ou_all_data_var, ou_all_T, ou_all_numTrials =  abcTau.preprocessing.extract_stats(ou_all, deltaT, binSize,
#                                                                                   summStat_metric, ifNorm, maxTimeLag)

print('data_sumStat {}, data_mean {}, data_var {}, T {}, numTrials {}'.format(data_sumStat, data_mean, data_var, T, numTrials))

ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,19,20), data_sumStat, maxfev=5000)
print(ou_all_popt)
ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

data_sumStat [ 1.00000000e+00  5.39026731e-01  3.01175887e-01  1.74050285e-01
  1.00660631e-01  5.62569172e-02  3.25571885e-02  2.19457135e-02
  1.14701791e-02  4.85115094e-03  3.17066012e-03  5.04209629e-05
 -3.11343603e-03  7.39495636e-04  4.57300007e-03  5.32260338e-03
  5.02920010e-03 -8.45639803e-04 -1.48969201e-03  2.67018792e-04], data_mean 2.2703611111111113, data_var 10.285099314043208, T 1800000, numTrials 1
[0.99478033 1.68961454]
84.48072696828667
